<a href="https://colab.research.google.com/github/joomj2000/Kobert_sentiment_analysis/blob/main/sentiment_analysis_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#필요 패키지 설치
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install pandas
!pip install kss==5.0
!pip install numpy==1.23.1


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 23.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661754 sha256=1f3298c8f1719928e71afb5bc92016d710c2531c6996e0e66481455fb59c7054
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86

In [ ]:
#kobert 공식 깃에서 토크나이저 관련 파일을 받기
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-55gdfytb/kobert-tokenizer_179e844bda4142cba205468723bc3234
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-55gdfytb/kobert-tokenizer_179e844bda4142cba205468723bc3234
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=c3445ac4285d3b2d0802fd04321694d497212e6dc2af971b6dff17d6410ac6b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-_dsg_bqa/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
#  Hugging Face를 통한 모델 및 토크나이저 Import
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 설정
device = torch.device("cuda:0")

In [ ]:
#데이터셋 로드
import pandas as pd
chatbot_data = pd.read_excel('/content/8개의 감정데이터셋_v9_2.xlsx')

In [ ]:
# 데이터셋 라벨링

chatbot_data.loc[(chatbot_data['Emotion'] == "우울한"), 'Emotion'] = 0
chatbot_data.loc[(chatbot_data['Emotion'] == "기쁜"), 'Emotion'] = 1
chatbot_data.loc[(chatbot_data['Emotion'] == "화가 나는"), 'Emotion'] = 2
chatbot_data.loc[(chatbot_data['Emotion'] == "슬픈"), 'Emotion'] = 3
chatbot_data.loc[(chatbot_data['Emotion'] == "편안한"), 'Emotion'] = 4
chatbot_data.loc[(chatbot_data['Emotion'] == "걱정스러운"), 'Emotion'] = 5
chatbot_data.loc[(chatbot_data['Emotion'] == "신이 난"), 'Emotion'] = 6
chatbot_data.loc[(chatbot_data['Emotion'] == "충만한"), 'Emotion'] = 7



In [ ]:

data_list = []
for q, label in zip(chatbot_data['Sentence'], chatbot_data['Emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size=0.25, random_state=0)


In [ ]:
print(len(dataset_train))
print(len(dataset_test))

170800
56934


In [ ]:
## Setting parameters
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 7
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [ ]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')



In [ ]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')


data_train = BERTDataset(dataset_train, 0, 1, tokenizer, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=8,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#정의한 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)
#model = BERTClassifier(bertmodel,  dr_rate=0.5).to('cpu')

# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01}, #델의 파라미터들을 그룹으로 나누어 옵티마이저에 전달하기 위
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
train_dataloader

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))

    model.eval()
    # for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    #     token_ids = token_ids.long().to(device)
    #     segment_ids = segment_ids.long().to(device)
    #     valid_length= valid_length
    #     label = label.long().to(device)
    #     out = model(token_ids, valid_length, segment_ids)
    #     test_acc += calc_accuracy(out, label)
    # print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)

      # 테스트 데이터가 잘못된 경우
       # if not torch.equal(out.argmax(dim=1), label):
          # 틀린 문장 출력
         # print("예측:", out.argmax(dim=1))
          #print("정답:", label)
          # print("틀린 문장:", token_ids)
          # print("틀린 문장 길이:", valid_length)

    print("epoch {} test acc {}".format(e + 1, test_acc / (batch_id + 1)))


<ipython-input-23-8cb76651f522>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1335 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.1135473251342773 train acc 0.1484375
epoch 1 batch id 201 loss 1.5470491647720337 train acc 0.23806747512437812
epoch 1 batch id 401 loss 0.8063300251960754 train acc 0.4076137780548628
epoch 1 batch id 601 loss 0.8625342845916748 train acc 0.509034421797005
epoch 1 batch id 801 loss 0.5162695050239563 train acc 0.5662550717852685
epoch 1 batch id 1001 loss 0.5718096494674683 train acc 0.6052229020979021
epoch 1 batch id 1201 loss 0.611430823802948 train acc 0.6321815154038302
epoch 1 train acc 0.6472787921348314


<ipython-input-23-8cb76651f522>:32: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/445 [00:00<?, ?it/s]

...

epoch 6 test acc 0.943288719982375


  0%|          | 0/1335 [00:00<?, ?it/s]

epoch 7 batch id 1 loss 0.07535934448242188 train acc 0.984375
epoch 7 batch id 201 loss 0.015856433659791946 train acc 0.9831700870646766
epoch 7 batch id 401 loss 0.032510776072740555 train acc 0.9819591645885287
epoch 7 batch id 601 loss 0.02780693583190441 train acc 0.9822821339434277
epoch 7 batch id 801 loss 0.016737787052989006 train acc 0.9826486423220974
epoch 7 batch id 1001 loss 0.06705408543348312 train acc 0.9832433191808192
epoch 7 batch id 1201 loss 0.030369123443961143 train acc 0.9838350853455454
epoch 7 train acc 0.9839185393258427


  0%|          | 0/445 [00:00<?, ?it/s]

epoch 7 test acc 0.9446625771094954


모델 저장

In [ ]:
print(model)

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [ ]:
'''
import torch

# 모델을 저장할 경로 및 파일 이름 지정
output_model_file = "model.bin"

# 모델 전체를 저장
torch.save(model, output_model_file)
'''

#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# 현재경로 /content/

import os
os.chdir('/content/drive/MyDrive/models/')
os.getcwd()

path = '/content/drive/MyDrive/models/'
torch.save(model, path + '0514_model.bin')  # 전체 모델 저장

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#구글드라이브 연동
from google.colab import drive
drive.mount('/content/drive')

# 현재경로 /content/

import os
os.chdir('/content/drive/MyDrive/models/')
os.getcwd()

path = '/content/drive/MyDrive/models/'
torch.save(model, path + 'sentiment_analysis_model_epoch10.pt')  # 전체 모델 저장

torch.save(model.state_dict(), 'sentiment_analysis_model_state_dict_epoch10.pt')  # 모델 객체의 state_dict 저장

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'sentiment_analysis_model_all_epoch10.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

import os
os.chdir('/content/drive/MyDrive/models/')

model1 = torch.load('sentiment_analysis_model.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model1.load_state_dict(torch.load('sentiment_analysis_model_state_dict.pt'))  # state_dict를 불러 온 후, 모델에 저장

checkpoint = torch.load('sentiment_analysis_model_all.tar')   # dict 불러오기
model1.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

결과 출력

In [ ]:
def find_most_frequent(lst):
    # 감정 라벨
    labels = ['우울', '기쁨', '화남', '슬픔', '편안', '걱정', '신남', '충만']

    # 리스트를 3등분
    chunk_size = len(lst[0]) // 3
    chunks = [lst[i:i+chunk_size] for i in range(0, len(lst), chunk_size)]

    result = []

    # 각 부분에서 확률값을 합산하여 가장 높은 라벨 찾기
    for chunk in chunks:
        # 각 라벨별로 확률값 합산
        label_sum = [0] * len(chunk[0])  # 라벨의 개수에 따라 초기화
        for probs in chunk:
            for i, prob in enumerate(probs):
                label_sum[i] += prob

        # 가장 높은 합산값을 가진 라벨 찾기
        most_common_label = label_sum.index(max(label_sum))

        # 가장 높은 합산값을 가진 라벨을 감정으로 변환하여 결과에 추가
        result.append(labels[most_common_label])

    # 결과 출력
    print(' '.join(result))


In [ ]:
import torch.nn.functional as F

def predict(predict_sentence):
    result = ''

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

    model1.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model1(token_ids, valid_length, segment_ids)

        # 소프트맥스 함수를 적용하여 확률값 계산
        probabilities = F.softmax(out, dim=1)

        for prob in probabilities:
            print("확률값:", prob)
            print("가장 높은 확률 클래스:", torch.argmax(prob).item())

        # # 가장 높은 확률을 가진 클래스 출력
        # if torch.argmax(probabilities) == 0:
        #     result = "우울"
        #     test_eval = "우울한"
        # elif torch.argmax(probabilities) == 1:
        #     result = "기쁨"
        #     test_eval = "기쁜"
        # elif torch.argmax(probabilities) == 2:
        #     result = "화남"
        #     test_eval = "화가 나는"
        # elif torch.argmax(probabilities) == 3:
        #     result = "슬픔"
        #     test_eval = "슬픈"
        # elif torch.argmax(probabilities) == 4:
        #     result = "편안"
        #     test_eval = "편안한"
        # elif torch.argmax(probabilities) == 5:
        #     result = "걱정"
        #     test_eval = "걱정스러운"
        # elif torch.argmax(probabilities) == 6:
        #     result = "신남"
        #     test_eval = "신이 난"
        # elif torch.argmax(probabilities) == 7:
        #     result = "충만"
        #     test_eval = "충만한"

        # print(test_eval + " 하루를 보냈네요.")
    return prob




In [ ]:
#!pip install kss --no-deps
#!pip uninstall kss
!pip install kss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 16.0 MB/s eta 0:00:00


In [ ]:
import kss
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

#질문 무한반복하기! 0 입력시 종료

while True:
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    result=[]
    for sent in kss.split_sentences(sentence):
      print(sent)
      result.append(predict(sent))
    print(result)
    find_most_frequent(result)

ModuleNotFoundError: No module named 'jamo'